In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
sys.path.append('/home/moritz/Desktop/programming/viewer')
from viewer.viewer_src import *
from utils.montages import full_bipolar_montage

In [57]:
def get_sample2(df,target):
    # Calculate absolute difference between col1 and target 
    diff = (df['preds'] - target).abs()
    # Find index of minimum difference 
    idx = diff.idxmin()
    # Get row for minimum difference 
    row = df.loc[idx]
    return row

In [58]:
def get_sample(df,low,high,n):
    return df[(df.preds>low)&(df.preds<=high)].iloc[0:n]

In [59]:
def get_samples(df,n):
    samples = []
    for target in np.arange(0,1.001,1/n):
        sample = get_sample2(df,target)
        samples.append(sample)
    df = pd.DataFrame(samples)
    df.reset_index(inplace=True,drop=True)
    return df

In [60]:
def load_and_cut_signal(path_signal,start,end,Fq):
    signal = np.load(path_signal)
    signal=signal[:,start*Fq:end*Fq]
    return signal

In [61]:
def filter_spikes(label,df):
    positive = df.Spike ==1
    negative = df.Spike == 0
    if label =='positive':
        df = df[positive].sort_values('preds',ascending=False)
    elif label =='negative':
        df = df[negative].sort_values('preds',ascending=False)
    return df

In [62]:
def build_viewer_module(ax,montage_module,x_start,x_end,Fq):
    y_labels=montage_module.montages[0].y_labels
    y_locations=montage_module.montages[0].y_locations
    viewer_module = init_viewer_module(ax,y_locations,y_labels,x_start=x_start,x_end=x_end,Fq=Fq)
    return viewer_module

In [63]:
def add_signal(path_signal,viewer_modules,montage_module,start,end,Fq):
    signal = load_and_cut_signal(path_signal,start,end,Fq)
    signal = montage_module.montages[0](signal)
    update_channels_with_signal(signal=signal,channel_lines=viewer_modules[i].channel_lines,y_locations=viewer_modules[i].y_locations) 


In [64]:
def process_ticks_and_title(ax,i,title):
    if i !=0:
        ax.set_yticks([])
    ax.set_xticks([0,1,2,3])
    ax.set_xlabel('time [s]')
    ax.set_title(title)
    return ax

In [65]:
def init_info(label,n,df):
    sub_df = get_samples(df,n)
    path_files = '/media/moritz/Expansion/Data/Spikes_clemson_10s/preprocessed_npy/'
    path_signals=list(path_files + sub_df.event_file+'.npy')
    titles = list(sub_df.preds.round(2).astype(str))
    subtitles = list(sub_df.event_file)
    Fq = 128
    return path_signals, titles, subtitles,Fq

In [66]:
n = 20
x_start,x_end = 0,3
signal_start,signal_end = 6,9
df = pd.read_csv('../../Models/generalized_all_ref_loc/results_localized_Clemson.csv')
df_loc = pd.read_excel('/media/moritz/Expansion/Data/Spikes_clemson_10s/tables/segments_labels_channels_montage.xlsx')
df = df.merge(df_loc[['event_file','Spike']])
df = df[df.ChannelLocation=='general']
sample_type = 'positive'
# init info 
df = filter_spikes(sample_type,df)
path_signals, titles, subtitles,Fq = init_info(sample_type,n=n,df=df)
# init figurea
fig, axs = plt.subplots(1,n+1,figsize = (20,5))
# init montage and viewer
montage_module = init_montage_module(montages= [full_bipolar_montage])
viewer_modules = []
# create figure
for i in range(n+1):
    viewer_module = build_viewer_module(axs[i],montage_module,x_start=x_start,x_end=x_end,Fq=Fq)
    viewer_modules.append(viewer_module)
    add_signal(path_signals[i],viewer_modules,montage_module,signal_start,signal_end,Fq)
    process_ticks_and_title(axs[i],i,titles[i])
fig.tight_layout()
fig.savefig(f'{sample_type}.png')

TypeError: 'NoneType' object is not subscriptable

In [67]:
df